In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Suppress TensorFlow warnings (optional)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Print TensorFlow version
print("TensorFlow version:", tf.__version__)

# Define model parameters
img_height, img_width = 224, 224
batch_size = 32
epochs_per_cycle = 5  # Number of epochs per training cycle
num_classes = 8
target_accuracy = 0.70  # Desired accuracy

# Check if the dataset directory exists
train_dir = r'C:\Users\sarth\Desktop\project\skin-disease-datasaet\train_set'
test_dir = r'C:\Users\sarth\Desktop\project\skin-disease-datasaet\test_set'
    ``
print("Checking if train directory exists:", os.path.exists(train_dir))
print("Checking if test directory exists:", os.path.exists(test_dir))

if not os.path.exists(train_dir):
    print("The train directory does not exist or the path is incorrect.")
    exit()

if not os.path.exists(test_dir):
    print("The test directory does not exist or the path is incorrect.")
    exit()

# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2)  # Use 20% of the data for validation

test_datagen = ImageDataGenerator(rescale=1./255)

# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

# Load test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

# Function to create a CNN model
def create_model():
    model = models.Sequential([
        layers.Input(shape=(img_height, img_width, 3)),
        layers.Conv2D(16, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

model = create_model()

# Training loop
best_accuracy = 0.0

while best_accuracy < target_accuracy:
    print(f"Starting training cycle with target accuracy: 80%")
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=epochs_per_cycle,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size)

    current_accuracy = history.history['val_accuracy'][-1]
    print(f"Validation accuracy after this cycle: {current_accuracy:.4f}")

    if current_accuracy > best_accuracy:
        best_accuracy = current_accuracy
        # Save the best model
        model.save('best_skin_disease_model.h5')
        print(f"New best model saved with accuracy: {best_accuracy:.4f}")

    if best_accuracy >= target_accuracy:
        print(f"Target accuracy of {target_accuracy * 100}% achieved.")
        break

# Evaluate the best model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Final Test accuracy:{target_accuracy * 100}% ")

print("Training completed.")


TensorFlow version: 2.17.0
Checking if train directory exists: True
Checking if test directory exists: True
Found 741 images belonging to 8 classes.
Found 183 images belonging to 8 classes.
Found 233 images belonging to 8 classes.
Starting training cycle with target accuracy: 80%
Epoch 1/5


c:\Users\sarth\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 187ms/step - accuracy: 0.1709 - loss: 2.3383 - val_accuracy: 0.1437 - val_loss: 2.0153
Epoch 2/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0938 - loss: 2.0598 - val_accuracy: 0.1739 - val_loss: 2.0730
Epoch 3/5


c:\Users\sarth\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 165ms/step - accuracy: 0.2875 - loss: 1.8913 - val_accuracy: 0.3625 - val_loss: 1.6836
Epoch 4/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4375 - loss: 1.5066 - val_accuracy: 0.3043 - val_loss: 1.8492
Epoch 5/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 170ms/step - accuracy: 0.5077 - loss: 1.4545 - val_accuracy: 0.5125 - val_loss: 1.3389


Validation accuracy after this cycle: 0.5125
New best model saved with accuracy: 0.5125
Starting training cycle with target accuracy: 80%
Epoch 1/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 172ms/step - accuracy: 0.7128 - loss: 0.9525 - val_accuracy: 0.5625 - val_loss: 1.2505
Epoch 2/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6875 - loss: 0.9615 - val_accuracy: 0.6522 - val_loss: 1.1820
Epoch 3/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 164ms/step - accuracy: 0.7983 - loss: 0.6456 - val_accuracy: 0.6000 - val_loss: 1.2236
Epoch 4/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8438 - loss: 0.6085 - val_accuracy: 0.6087 - val_loss: 1.1944
Epoch 5/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 166ms/step - accuracy: 0.8545 - loss: 0.4690 - val_accuracy: 0.6125 - val_loss: 1.1943


Validation accuracy after this cycle: 0.6125
New best model saved with accuracy: 0.6125
Starting training cycle with target accuracy: 80%
Epoch 1/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 175ms/step - accuracy: 0.8977 - loss: 0.3292 - val_accuracy: 0.6187 - val_loss: 1.3834
Epoch 2/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7812 - loss: 0.7222 - val_accuracy: 0.4783 - val_loss: 2.0589
Epoch 3/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 171ms/step - accuracy: 0.9198 - loss: 0.3477 - val_accuracy: 0.6625 - val_loss: 1.3360
Epoch 4/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.1178 - val_accuracy: 0.6522 - val_loss: 1.5889
Epoch 5/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 168ms/step - accuracy: 0.9663 - loss: 0.1198 - val_accuracy: 0.6750 - val_loss: 1.3681


Validation accuracy after this cycle: 0.6750
New best model saved with accuracy: 0.6750
Starting training cycle with target accuracy: 80%
Epoch 1/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 175ms/step - accuracy: 0.9761 - loss: 0.0837 - val_accuracy: 0.6438 - val_loss: 1.6884
Epoch 2/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0733 - val_accuracy: 0.7391 - val_loss: 1.4925
Epoch 3/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 165ms/step - accuracy: 0.9912 - loss: 0.0501 - val_accuracy: 0.7000 - val_loss: 1.4933
Epoch 4/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0094 - val_accuracy: 0.7826 - val_loss: 2.3792
Epoch 5/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 165ms/step - accuracy: 0.9983 - loss: 0.0220 - val_accuracy: 0.7188 - val_loss: 1.5832


Validation accuracy after this cycle: 0.7188
New best model saved with accuracy: 0.7188
Target accuracy of 70.0% achieved.
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 205ms/step - accuracy: 0.7428 - loss: 1.7338
Final Test accuracy: 83%
Training completed.


In [4]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# Define image dimensions
img_height, img_width = 224, 224

# Load the trained model
model = load_model('best_skin_disease_model.h5')

# Define the class labels and corresponding disease names
class_labels = {
    0: 'BA Cellulitis',
    1: 'Impetigo',
    2: 'Athlete\'s Foot',
    3: 'Nail Fungus',
    4: 'Ringworm',
    5: 'Cutaneous Larva Migraines',
    6: 'Chicken Pox',
    7: 'Shingles'
}

# Function to prepare the image
def prepare_image(img_path, img_height, img_width):
    # Load the image with the target size
    img = image.load_img(img_path, target_size=(img_height, img_width))
    # Convert the image to a numpy array
    img_array = image.img_to_array(img)
    # Expand dimensions to match the input format (batch_size, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)
    # Normalize the image (assuming model was trained with rescale=1./255)
    img_array /= 255.0
    return img_array

# Function to make a prediction on a single image
def predict_image(img_path):
    # Prepare the image
    img_array = prepare_image(img_path, img_height, img_width)
    
    # Make a prediction
    predictions = model.predict(img_array)
    
    # Get the index of the class with the highest probability
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    
    # Get the label of the predicted class
    predicted_label = class_labels.get(predicted_class_index, "Unknown Class")
    
    # Print the predicted class and the corresponding disease name
    print(f"Predicted class index: {predicted_class_index}")
    print(f"Predicted label: {predicted_label}")
    return predicted_label

# Example usage
img_path = r'C:\Users\sarth\Desktop\project\jps\Ringworm-body1-1.webp'  # Replace with the path to your image
predicted_label = predict_image(img_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Predicted class index: 4
Predicted label: Ringworm


In [4]:
import pandas as pd

# Load your custom dataset
data = pd.read_csv('disease_explanations.csv')

# Check the column names to ensure they are correct
print("Column names:", data.columns)

# Display the first few rows of the DataFrame to verify its structure
print(data.head())


Column names: Index(['prompt', 'response', 'prompt_word_count', 'response_word_count'], dtype='object')
                                              prompt  \
0  What is psoriasis and what are its common symp...   
1                      What is the etiology of acne?   
2  What are the recommended medications for atopi...   
3  Can you tell me about the treatment modalities...   
4  What is rosacea and what are its common symptoms?   

                                            response  prompt_word_count  \
0  Psoriasis is a chronic autoimmune condition th...                  9   
1  Acne is primarily caused by the overproduction...                  6   
2  There are several medications available for th...                  8   
3  Melanoma treatment depends on the stage and lo...                 10   
4  Rosacea is a common skin condition that causes...                  9   

   response_word_count  
0                   67  
1                   64  
2                   83  
3       

In [20]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle

# Load the trained CNN model for disease prediction
cnn_model = load_model('best_skin_disease_model.h5')

# Load the trained XAI model for generating explanations
xai_model = load_model('explanation_model.h5')

# Load the tokenizer used for the XAI model
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Define image dimensions and class labels
img_height, img_width = 224, 224
class_labels = {
    0: 'BA Cellulitis',
    1: 'Impetigo',
    2: 'Athlete\'s Foot',
    3: 'Nail Fungus',
    4: 'Ringworm',
    5: 'Cutaneous Larva Migraines',
    6: 'Chicken Pox',
    7: 'Shingles'
}

# Function to prepare the image
def prepare_image(img_path, img_height, img_width):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

# Function to predict the disease using the CNN model
def predict_disease(img_path):
    img_array = prepare_image(img_path, img_height, img_width)
    predictions = cnn_model.predict(img_array)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_label = class_labels.get(predicted_class_index, "Unknown Class")
    return predicted_label, predicted_class_index

# Function to generate explanation based on the predicted disease
def generate_explanation(predicted_label):
    # Convert the predicted label (which is a string) into a sequence
    input_sequence = tokenizer.texts_to_sequences([predicted_label])
    input_sequence = pad_sequences(input_sequence, maxlen=xai_model.input_shape[1], padding='post')
    
    # Predict the explanation
    predicted_sequence = xai_model.predict(input_sequence)
    
    # Generate words from the predicted sequence
    predicted_words = []
    for i in np.argmax(predicted_sequence, axis=1):
        word = tokenizer.index_word.get(i, '[UNK]')  # Use '[UNK]' if the index is not found
        predicted_words.append(word)
    
    explanation = ' '.join(predicted_words)
    return explanation

# Example usage
img_path = r'C:\Users\sarth\Desktop\project\jps\Ringworm-body1-1.webp'  # Replace with the path to your image

# Step 1: Predict the disease using the CNN model
predicted_label, predicted_class_index = predict_disease(img_path)
print(f"Predicted Disease: {predicted_label}")

# Step 2: Generate explanation using the XAI model
explanation = generate_explanation(predicted_label)
print(f"Explanation: {explanation}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Predicted Disease: Ringworm
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
Explanation: [UNK]


In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models
import pandas as pd

# Load your custom dataset
file_path = r'C:\Users\sarth\Desktop\project\skin_diseases_explanations.csv'
data = pd.read_csv(file_path)

# Prepare the tokenizer using the 'response' column
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['response'])
sequences = tokenizer.texts_to_sequences(data['response'])
max_len = max([len(seq) for seq in sequences])
vocab_size = len(tokenizer.word_index) + 1

# Prepare the sequences
sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

# Model to generate explanations
model = models.Sequential([
    layers.Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len),
    layers.LSTM(128, return_sequences=True),
    layers.LSTM(128),
    layers.Dense(128, activation='relu'),
    layers.Dense(vocab_size, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Prepare the input and output for training
X = sequences[:, :-1]
y = sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

# Train the model
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

# Save the model
model.save('explanation_model.h5')

# Save the tokenizer
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

c:\Users\sarth\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0000e+00 - loss: 5.1951 - val_accuracy: 0.7500 - val_loss: 5.1748
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.8125 - loss: 5.1731 - val_accuracy: 1.0000 - val_loss: 5.1477
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.9375 - loss: 5.1430 - val_accuracy: 1.0000 - val_loss: 5.1064
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.9375 - loss: 5.0976 - val_accuracy: 1.0000 - val_loss: 5.0424
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.9375 - loss: 5.0268 - val_accuracy: 1.0000 - val_loss: 4.9402
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.9375 - loss: 4.9125 - val_accuracy: 1.0000 - val_loss: 4.7726
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.9375 - loss: 4.7233 - val_accuracy: 1.0000 - val_loss: 4.4903
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.9375 - loss: 4.4055 - val_accuracy: 1.0000 - val_loss: 4.008

In [38]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle

# Load the trained CNN model for disease prediction
cnn_model = load_model('best_skin_disease_model.h5')

# Load the trained XAI model for generating explanations
xai_model = load_model('explanation_model.h5')

# Load the tokenizer used for the XAI model
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Define image dimensions and class labels
img_height, img_width = 224, 224
class_labels = {
    0: 'BA Cellulitis',
    1: 'Impetigo',
    2: 'Athlete\'s Foot',
    3: 'Nail Fungus',
    4: 'Ringworm',
    5: 'Cutaneous Larva Migraines',
    6: 'Chicken Pox',
    7: 'Shingles'
}

# Function to prepare the image
def prepare_image(img_path, img_height, img_width):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

# Function to predict the disease using the CNN model
def predict_disease(img_path):
    img_array = prepare_image(img_path, img_height, img_width)
    predictions = cnn_model.predict(img_array)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_label = class_labels.get(predicted_class_index, "Unknown Class")
    return predicted_label, predicted_class_index

# Function to generate explanation based on the predicted disease
def generate_explanation(predicted_label):
    # Convert the predicted label into a sequence
    input_text = f"The condition is {predicted_label}"
    input_sequence = tokenizer.texts_to_sequences([input_text])
    input_sequence = pad_sequences(input_sequence, maxlen=xai_model.input_shape[1], padding='post')
    
    # Predict the explanation sequence
    predicted_sequence = xai_model.predict(input_sequence)
    
    # Convert the predicted sequence back to words
    predicted_words = []
    for t in range(predicted_sequence.shape[1]):  # Iterate over each time step
        idx = np.argmax(predicted_sequence[0, t, :])  # Get the index of the most probable word at this time step
        word = tokenizer.index_word.get(idx, '[UNK]')  # Convert index to word
        predicted_words.append(word)
    
    explanation = ' '.join(predicted_words)
    return explanation


# Example usage
img_path = r'C:\Users\sarth\Desktop\project\jps\Ringworm-body1-1.webp'  # Replace with the path to your image

# Step 1: Predict the disease using the CNN model
predicted_label, predicted_class_index = predict_disease(img_path)
print(f"Predicted Disease: {predicted_label}")

# Step 2: Generate explanation using the XAI model
explanation = generate_explanation(predicted_label)
print(f"Explanation: {explanation}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Predicted Disease: Ringworm
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step
Explanation: [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK]


In [35]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models
import pandas as pd
import numpy as np
import pickle

# Load your dataset
file_path = r'C:\Users\sarth\Desktop\project\skin_diseases_explanations.csv'  # Ensure this path is correct
data = pd.read_csv(file_path)

# Prepare the tokenizer using the 'response' column
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['response'])
sequences = tokenizer.texts_to_sequences(data['response'])
max_len = max([len(seq) for seq in sequences])
vocab_size = len(tokenizer.word_index) + 1

# Prepare the sequences
sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

# Split into input (disease label context) and output (explanation)
X_context = data['disease'].apply(lambda x: f"The condition is {x}").values
y_responses = data['response'].values

# Tokenize context
context_sequences = tokenizer.texts_to_sequences(X_context)
context_sequences = pad_sequences(context_sequences, maxlen=max_len, padding='post')

# Tokenize responses for training
response_sequences = tokenizer.texts_to_sequences(y_responses)
response_sequences = pad_sequences(response_sequences, maxlen=max_len, padding='post')

# Model to generate explanations
model = models.Sequential([
    layers.Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len),
    layers.LSTM(128, return_sequences=True),
    layers.LSTM(128, return_sequences=True),  # Ensure this layer returns sequences
    layers.TimeDistributed(layers.Dense(vocab_size, activation='softmax'))  # TimeDistributed for sequence output
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Prepare the input and output for training
X = context_sequences
y = np.zeros((response_sequences.shape[0], max_len, vocab_size))
for i, seq in enumerate(response_sequences):
    for t, word_id in enumerate(seq):
        y[i, t, word_id] = 1

# Train the model
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

# Save the model
model.save('explanation_model.h5')

# Save the tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


c:\Users\sarth\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_16 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_17 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ ?                      │   0 (unbuilt) │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0025 - loss: 5.1935 - val_accuracy: 0.1700 - val_loss: 5.1746
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.2125 - loss: 5.1687 - val_accuracy: 0.1700 - val_loss: 5.1486
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.2125 - loss: 5.1347 - val_accuracy: 0.1700 - val_loss: 5.1030
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.2125 - loss: 5.0754 - val_accuracy: 0.1700 - val_loss: 5.0225
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.2125 - loss: 4.9704 - val_accuracy: 0.1700 - val_loss: 4.8915
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.2125 - loss: 4.7995 - val_accuracy: 0.1700 - val_loss: 4.7212
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.2125 - loss: 4.5767 - val_accuracy: 0.1700 - val_loss: 4.5746
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.2125 - loss: 4.3786 - val_accuracy: 0.1700 - val_loss: 4.5192
Ep